<a href="https://colab.research.google.com/github/andreytsimbalov/Diplom/blob/main/data_news_corrector_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sklearn.cluster as sc
import gensim
import random
import pandas as pd
import numpy as np
import time
import datetime
from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt

md = '/content/drive/My Drive/Colab Notebooks/Diplom/'

In [ ]:
months = {
    'января':'01',
    'февраля':'02',
    'марта':'03',
    'апреля':'04',
    'мая':'05',
    'июня':'06',
    'июля':'07',
    'августа':'08',
    'сентября':'09',
    'октября':'10',
    'ноября':'11',
    'декабря':'12'
}
months_numbers = list(months.values())

In [ ]:
def show_elbow_plot(model,vocab_m):
  scores = []
  for n_cl in tqdm(range(1,10)):
    km = sc.KMeans(init='k-means++',n_clusters=n_cl, random_state=0).fit(model.wv[vocab_m])
    scores+= [-km.score(model.wv[vocab_m])]

  plt.scatter(range(1,10),scores)
  plt.show()

In [ ]:
def take_from_model(model):
  vocab_m = np.array(list(model.wv.vocab))
  counts_m = np.array([model.wv.vocab[v].count for v in vocab_m])
  n=10 # Выводит первые n самых частотных слов
  counts_max_m = sorted(np.unique(sorted(counts_m,reverse=True)[:n]),reverse=True)
  freq_index = [j  for i in counts_max_m for j in np.where(counts_m == i)[0]]
  freq_m = [vocab_m[i] for i in freq_index]
  return vocab_m, counts_m, counts_max_m, freq_m

In [ ]:
def correct_text_news(text):
    try:
        if text.find('РИА Новости')!=-1:
            text_start = text.find('РИА Новости')+len('РИА Новости')+2
            text = text[text_start:]
    except:
        print(text)
    return text

def convert_to_datetime(date,time):
    d = date.split()
    # d = d[0] + '-' + months[d[1]] +  '-' + d[2]
    d = d[2] + '-' + months[d[1]] +  '-' + d[0]
    # print(d)
    return pd.to_datetime(d + ' '+ time + ':00')

def take_list_from_tags(tags_str):
    s = tags_str[1:-1]
    s = s.replace(' ','_')
    s = s.replace("',_'",' ')
    s = s.replace("'",'')
    s = s.split()
    return s

# Обработка новостей RIA 2020

In [ ]:
data = pd.read_pickle(md+'data/news_ria_2020')

In [ ]:
def get_data_with_filename(filename):
  # data = pd.read_csv(file_name).drop('Unnamed: 0',axis=1)
  data = pd.read_pickle(filename)
  data.dropna(subset=['text'], inplace=True)
  data['text'] = [correct_text_news(data.loc[i,'text']) for i in range(len(data))]
  data = data.iloc[np.where(np.array(data.text!=['' for _ in data.text]))[0]]

  # print(len(data))
  data.index = np.arange(len(data))
  data = data.iloc[np.where(np.array(data.text!=['' for _ in data.text]))[0]]
  # print(len(data))

  data['dt'] = [convert_to_datetime(data.date[i], data.time[i]) for i in range(len(data))]
  data['main_text'] = data.headline + '. '+ data.text
  data = data.drop(['date','time', 'link', 'headline', 'text','views'],axis=1)
  data = data.sort_values('dt',ignore_index=True)
  
  return data

In [ ]:
# for i in tqdm(range(0,len(months_numbers))):
#   d = get_data_with_filename(md+'data/news_'+months_numbers[i]+'_2020')
#   print(d.dt.head(1))
#   print(d.dt.tail(1))

In [ ]:
# datas = [get_data_with_filename(md+'data/news_'+months_numbers[i]+'_2020') for i in range(len(months_numbers))]

data = get_data_with_filename(md+'data/news_'+months_numbers[0]+'_2020') 
for i in tqdm(range(1,len(months_numbers))):
  data = data.append(get_data_with_filename(md+'data/news_'+months_numbers[i]+'_2020'))
  # print(i)
data.index = np.arange(len(data))
data

,views,tags,dt,main_text
0,12388,"[Новый год—2020, Парковка, Москва, Общество]",2020-01-01 00:33:00,Парковка в Москве в новогодние праздники будет...
1,1029790,"[Новый год—2020, Шоу-бизнес, Анастасия Заворот...",2020-01-01 00:38:00,Анастасия Заворотнюк поблагодарила россиян за ...
2,137369,"[Культура, Россия, Шоу-бизнес, Юрий Лоза, Нико...",2020-01-01 00:57:00,Лоза отреагировал на критику концертов Пугачев...
3,24294,"[Угрозы в адрес журналистов Sputnik Эстония, Е...",2020-01-01 01:08:00,Sputnik Эстония перешел на работу в чрезвычайн...
4,88263,"[Ситуация на Украине, Новый год, Владимир Зеле...",2020-01-01 01:19:00,Зеленский в новогоднем поздравлении призвал ук...
...,...,...,...,...
45924,455876,"[Оппозиционная платформа - За жизнь, Ситуация ...",2020-12-31 20:07:00,Депутат Рады надел шапку с красной звездой и о...
45925,29885,"[Домодедово (аэропорт), Шереметьево (аэропорт)...",2020-12-31 21:16:00,В Москве задержали или отменили около 105 рейс...
45926,42263,"[Россия, Снегурочка, Дед Мороз, Сургут, ДТП, П...",2020-12-31 21:42:00,В Сургуте Дед Мороз разбился в ДТП. Мужчина в ...
45927,40433,"[Распространение коронавируса, Панамериканская...",2020-12-31 23:19:00,ВОЗ внесла вакцину Pfizer в список для экстрен...


In [ ]:
data.to_pickle(md+'data/news_ria_2020')

In [ ]:
data.main_text[0]

NameError: ignored

# Обработка новостей Lenta 2020

In [ ]:
data = pd.read_pickle(md+'data/news_lenta_2020')

In [ ]:
tags = data.tag.to_numpy()
tags_un = np.unique(tags)

In [ ]:
data.tags.to_numpy()

array(['Экономика', 'Бывший СССР', 'Экономика', ..., 'Мир',
       'Наука и техника', 'Мир'], dtype=object)

In [ ]:
tags_un_counter = [len(data[data.tag==tu].text) for tu in tags_un]
tags_un_dict = dict(zip(tags_un, tags_un_counter))
tags_un_dict

{'69-я параллель': 44,
 'Бывший СССР': 616,
 'Дом': 345,
 'Из жизни': 421,
 'Интернет и СМИ': 452,
 'Культура': 348,
 'Мир': 967,
 'Наука и техника': 451,
 'Нацпроекты': 304,
 'Путешествия': 366,
 'Россия': 1655,
 'Силовые структуры': 427,
 'Спорт': 536,
 'Ценности': 349,
 'Экономика': 836}

In [ ]:
def get_data_lenta_with_filename(filename):
  # data = pd.read_csv(file_name).drop('Unnamed: 0',axis=1)
  data = pd.read_pickle(filename)
  data.dropna(subset=['text'], inplace=True)
  data = data[data.text!='']

  # print(len(data))
  data.index = np.arange(len(data))
  # data = data.iloc[np.where(np.array(data.text!=['' for _ in data.text]))[0]]
  # print(len(data))
  data.rename(columns={'tag': 'tags'}, inplace=True)
  for i in data.index:
    data['tags'].iloc[i] = [data['tags'].iloc[i]]


  data['dt'] = [convert_to_datetime(data.date[i], data.time[i]) for i in range(len(data))]
  data['main_text'] = data.headline + '. '+ data.text
  data = data.drop(['date','time', 'link', 'headline', 'text'],axis=1)
  data = data.sort_values('dt',ignore_index=True)
  
  return data
data = get_data_lenta_with_filename(md+'data/news_lenta_12_2020')
data

,tags,dt,main_text
0,[Россия],2020-12-01 00:05:00,Врач описал поражения центральной нервной сист...
1,[Россия],2020-12-01 00:09:00,В России внесли изменения в права и другие док...
2,[Наука и техника],2020-12-01 00:09:00,Nokia выпустит свой первый ноутбук. Nokia пред...
3,[Экономика],2020-12-01 00:13:00,Цены на нефть показали рекордные темпы роста с...
4,[Мир],2020-12-01 00:28:00,Иран заявил о приведшем к смерти физика-ядерщи...
...,...,...,...
8112,[Мир],2020-12-31 23:08:00,Трамп раскрыл срок поставок американской вакци...
8113,[Мир],2020-12-31 23:29:00,США укрепили военную базу в Ираке в преддверии...
8114,[Мир],2020-12-31 23:35:00,ВОЗ рекомендовала вакцину Pfizer для экстренно...
8115,[Наука и техника],2020-12-31 23:38:00,Рогозин назвал чушью версию о причастности роб...


In [ ]:
# datas = [get_data_with_filename(md+'data/news_'+months_numbers[i]+'_2020') for i in range(len(months_numbers))]

data = get_data_lenta_with_filename(md+'data/news_lenta_'+months_numbers[0]+'_2020') 
for i in tqdm(range(1,len(months_numbers))):
  data = data.append(get_data_lenta_with_filename(md+'data/news_lenta_'+months_numbers[i]+'_2020'))
  # print(i)
data.index = np.arange(len(data))
data

,tags,dt,main_text
0,[Экономика],2020-01-01 00:07:00,В России повысили зарплаты. В России повышен м...
1,[Бывший СССР],2020-01-01 00:27:00,Партия Порошенко потребовала запретить поставк...
2,[Экономика],2020-01-01 00:35:00,В России подорожали алкогольные напитки и сига...
3,[Наука и техника],2020-01-01 00:48:00,Рогозин назвал поклонников Маска «Свидетелями ...
4,[Россия],2020-01-01 01:02:00,В России изменились правила регистрации новых ...
...,...,...,...
86788,[Мир],2020-12-31 23:08:00,Трамп раскрыл срок поставок американской вакци...
86789,[Мир],2020-12-31 23:29:00,США укрепили военную базу в Ираке в преддверии...
86790,[Мир],2020-12-31 23:35:00,ВОЗ рекомендовала вакцину Pfizer для экстренно...
86791,[Наука и техника],2020-12-31 23:38:00,Рогозин назвал чушью версию о причастности роб...


In [ ]:
data.to_pickle(md+'data/news_lenta_2020')

# Обработка новостей RBC 2020

In [ ]:
data = pd.read_pickle(md+'data/news_rbc_01_2020')

In [ ]:
data.dropna(subset=['text', 'tag', 'other_tags', 'headline', 'time_int'], inplace=True)
data = data[data.text!='']

data.index = np.arange(len(data))

In [ ]:
tags = data.tag.to_numpy()
tags_un = np.unique(tags)

In [ ]:
tags_un_counter = [len(data[data.tag==tu].text) for tu in tags_un]
tags_un_dict = dict(zip(tags_un, tags_un_counter))
tags_un_dict

{'Бизнес': 27,
 'Общество': 1129,
 'Политика': 315,
 'Технологии и медиа': 57,
 'Финансы': 12,
 'Экономика': 27}

In [ ]:
def get_data_lenta_with_filename(filename):
  data = pd.read_pickle(filename)
  data.dropna(subset=['text', 'tag', 'other_tags', 'headline', 'time_int'], inplace=True)
  data = data[data.text!='']

  data.index = np.arange(len(data))
  data['tags'] = np.nan
  for i in data.index:
    data['tags'].iloc[i] = [data['tag'].iloc[i]] + data['other_tags'].iloc[i]


  data['dt'] = pd.to_datetime(data.time_int, unit='s')
  data['main_text'] = data.headline + '. '+ data.text
  data = data.drop(['link', 'headline', 'text', 'tag', 'time_in_str','time_int','other_tags'],axis=1)
  data = data.sort_values('dt',ignore_index=True)
  
  return data
data = get_data_lenta_with_filename(md+'data/news_rbc_10_2020')
data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,tags,dt,main_text
0,"[Политика, Борис Грызлов, Минские соглашения, ...",2020-10-01 00:22:21,Грызлов заявил о «фактическом прекращении» мин...
1,"[Общество, коронавирус, COVID-19]",2020-10-01 00:23:04,В Центре Гамалеи назвали главную ошибку зарази...
2,"[Общество, Роспотребнадзор, коронавирус, COVID...",2020-10-01 01:26:54,В Роспотребнадзоре уточнили сроки выхода Росси...
3,"[Общество, Киев, посольство, США, новости]",2020-10-01 02:59:47,Посольство США подтвердило гибель своей сотруд...
4,"[Политика, Дональд Трамп, Владимир Познер, выб...",2020-10-01 04:07:09,Познер назвал постыдными оскорбления Трампа и ...
...,...,...,...
1803,"[Общество, Франция, РПЦ, ислам]",2020-10-31 18:02:43,В РПЦ посоветовали Франции принять закон о защ...
1804,"[Общество, Челябинск, больница, взрыв, пожар]",2020-10-31 18:49:01,В Челябинске 13 пациентов больницы после взрыв...
1805,"[Общество, Турция, землетрясение]",2020-10-31 20:41:50,Число жертв землетрясения в Турции возросло до...
1806,"[Политика, Михаил Саакашвили, парламентские вы...",2020-10-31 21:32:39,Саакашвили отказался претендовать на пост прем...


In [ ]:
# datas = [get_data_with_filename(md+'data/news_'+months_numbers[i]+'_2020') for i in range(len(months_numbers))]

data = get_data_lenta_with_filename(md+'data/news_rbc_'+months_numbers[0]+'_2020') 
for i in tqdm(range(1,len(months_numbers))):
  data = data.append(get_data_lenta_with_filename(md+'data/news_rbc_'+months_numbers[i]+'_2020'))
  # print(i)
data.index = np.arange(len(data))
data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

,tags,dt,main_text
0,"[Общество, Анастасия Заворотнюк, актриса]",2020-01-01 02:43:00,Семья Заворотнюк в Instagram поблагодарила рос...
1,"[Политика, Дональд Трамп, Иран]",2020-01-01 10:29:24,Трамп заявил о нежелании вступать в военный ко...
2,"[Общество, сон, праздники, новости РБК, Новый ...",2020-01-01 11:00:20,Врачи предупредили об опасности бессонной ново...
3,"[Общество, Москва, пенсионерка, кража, новости...",2020-01-01 12:18:08,У пенсионерки украли сумку с более 10 млн руб....
4,"[Общество, новости РБК, диета, Новый год]",2020-01-01 13:00:33,Диетологи рассказали об опасности переедания в...
...,...,...,...
19149,"[Общество, Испания, Великобритания, Гибралтар,...",2020-12-31 19:20:50,Испания и Британия достигли соглашения по Гибр...
19150,"[Экономика, инфляция, Росстат, Минэкономразвит...",2020-12-31 20:07:19,Росстат оценил инфляцию в России по итогам 202...
19151,"[Общество, Великобритания, коронавирус, COVID-19]",2020-12-31 20:44:25,В Британии за сутки впервые выявили более 55 т...
19152,"[Общество, Ералаш, Борис Грачевский, коронавир...",2020-12-31 21:24:11,Супруга худрука «Ералаша» сообщила о состоянии...


In [ ]:
data.to_pickle(md+'data/news_rbc_2020')

# Создание общего датасета

In [ ]:
df_ria = pd.read_pickle(md+'data/news_ria_2020')
df_rbc = pd.read_pickle(md+'data/news_rbc_2020')
df_len = pd.read_pickle(md+'data/news_lenta_2020')

In [ ]:
df_ria['website'] = 'ria'
df_rbc['website'] = 'rbc'
df_len['website'] = 'lenta'

In [ ]:
data = df_ria
data = data.append(df_rbc, ignore_index = True)
data = data.append(df_len, ignore_index = True)

data.sort_values('dt',ignore_index=True, inplace=True)
print(len(data))
data.head()

151876


,tags,dt,main_text,website
0,[Экономика],2020-01-01 00:07:00,В России повысили зарплаты. В России повышен м...,lenta
1,[Бывший СССР],2020-01-01 00:27:00,Партия Порошенко потребовала запретить поставк...,lenta
2,"[Новый год—2020, Парковка, Москва, Общество]",2020-01-01 00:33:00,Парковка в Москве в новогодние праздники будет...,ria
3,[Экономика],2020-01-01 00:35:00,В России подорожали алкогольные напитки и сига...,lenta
4,"[Новый год—2020, Шоу-бизнес, Анастасия Заворот...",2020-01-01 00:38:00,Анастасия Заворотнюк поблагодарила россиян за ...,ria


In [ ]:
tags_len = data.tags.apply(lambda x: len(x))
tags_len_non_zero_pos = np.where(tags_len!=0)
data = data.iloc[tags_len_non_zero_pos]

In [ ]:
data.to_pickle(md+'data/news_main_2020')